# Exercise 3
### Anton Wiehe & Angelie Kraftelie

In [26]:
from PIL import Image
import numpy as np

# Test image
im_frame = Image.open("positives/" + 'p01.png')
np_frame = np.array(im_frame.getdata())
print(len(np_frame))

# Load images (30, 573, 3) == (num_images, num_pixels, rgb)
positives = np.zeros((30, 576, 3))
for i in range(30):
    im_frame = Image.open("positives/" + 'p' + ('0' if (i+1) < 10 else '') + str(i + 1) + '.png')
    positives[i] = np.array(im_frame.getdata())
    
negatives = np.zeros((30, 576, 3))
for i in range(30):
    im_frame = Image.open("negatives/" + 'n' + ('0' if (i+1) < 10 else '') + str(i + 1) + '.png')
    negatives[i] = np.array(im_frame.getdata())
    
means = np.mean(positives, axis=0)
print(means)

576
[[180.43333333 149.7        141.36666667]
 [181.         150.66666667 143.03333333]
 [180.83333333 151.43333333 142.53333333]
 ...
 [177.5        146.3        138.66666667]
 [177.76666667 145.3        138.06666667]
 [175.43333333 143.1        137.66666667]]


576